In [ ]:
import glob
import numpy as np
from PIL import Image, ImageEnhance
import torch
import torch.optim as optim
from torchvision import models
from torchvision import transforms as tf
import torch.nn.functional as F
import os

ROOT_PATH = os.path.dirname(os.path.abspath("__file__"))

IMAGE_FOLDER = 'folder with downloaded images'

STYLE = 'style image folder'

STYLE_IMAGE = 'image file'

OUT_FOLDER = 'name of desired output folder'

if not os.path.exists(os.path.join(ROOT_PATH, OUT_FOLDER)):
    os.mkdir(os.path.join(ROOT_PATH, OUT_FOLDER))

style_img = Image.open(os.path.join(ROOT_PATH, STYLE, STYLE_IMAGE)).convert('RGB')

vgg = models.vgg19(pretrained=True).features

for param in vgg.parameters():
    param.requires_grad_(False)

device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

vgg.to(device)

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

def transformation(img):
    tasks = tf.Compose([tf.Resize(500),
                        tf.ToTensor(),
                        tf.Normalize(mean, std)])

    img = tasks(img)

    img = img.unsqueeze(0)

    return img


style_img = transformation(style_img).to(device)


def tensor_to_image(tensor):
    image = tensor.clone().detach()

    image = image.cpu().numpy().squeeze()

    image = image.transpose(1, 2, 0)

    image *= np.array(std) + np.array(mean)

    image = image.clip(0, 1)

    return image


LAYERS_OF_INTEREST = {'0': 'conv1_1',
                      '5': 'conv2_1',
                      '10': 'conv3_1',
                      '19': 'conv4_1',
                      '21': 'conv4_2',
                      '28': 'conv5_1'}


def apply_model_and_extract_features(image, model):
    x = image

    features = {}

    for name, layer in model._modules.items():
        x = layer(x)

        if name in LAYERS_OF_INTEREST:
            features[LAYERS_OF_INTEREST[name]] = x

    return features


style_img_features = apply_model_and_extract_features(style_img, vgg)


def calculate_gram_matrix(tensor):
    _, channels, height, width = tensor.size()

    tensor = tensor.view(channels, height * width)

    gram_matrix = torch.mm(tensor, tensor.t())

    gram_matrix = gram_matrix.div(channels * height * width)

    return gram_matrix


style_features_gram_matrix = {layer: calculate_gram_matrix(style_img_features[layer]) for layer in style_img_features}

weights = {'conv1_1': 1.0, 'conv2_1': 0.75, 'conv3_1': 0.35,
           'conv4_1': 0.25, 'conv5_1': 0.15}

img_lst = []

# reads image filename from directory and appends to list
for name in glob.glob(os.path.join(ROOT_PATH, IMAGE_FOLDER) + "\*.jpg"):
    img_lst.append(name)

# sorts the list by numeric order to check output against input
img_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

count = 0

for image in img_lst:
    content_img = Image.open(image).convert('RGB')

    # enhancer = ImageEnhance.Brightness(content_img)
    #
    # factor = 1.3
    #
    # content_img = enhancer.enhance(factor)

    content_img = transformation(content_img).to(device)

    content_img_features = apply_model_and_extract_features(content_img, vgg)

    target = content_img.clone().requires_grad_(True).to(device)

    optimizer = optim.Adam([target], lr=0.003)

    for i in range(1, 1000):
        target_features = apply_model_and_extract_features(target, vgg)

        content_loss = F.mse_loss(target_features['conv4_2'], content_img_features['conv4_2'])

        style_loss = 0

        for layer in weights:
            target_feature = target_features[layer]

            target_gram_matrix = calculate_gram_matrix(target_feature)

            style_gram_matrix = style_features_gram_matrix[layer]

            layer_loss = F.mse_loss(target_gram_matrix, style_gram_matrix)

            layer_loss *= weights[layer]

            _, channels, height, width = target_feature.shape

            style_loss += layer_loss

        total_loss = 1000000 * style_loss + content_loss

        if i % 500 == 0:
            print('Epoch {}:, Style Loss : {:4f}, Content Loss : {:4f}'.format(i, style_loss, content_loss))

        optimizer.zero_grad()

        total_loss.backward()

        optimizer.step()

    count += 1

    out = tensor_to_image(target)

    img_out = Image.fromarray((out * 255).astype('uint8'), 'RGB')

    img_out.save(os.path.join(ROOT_PATH, OUT_FOLDER) + f"\image_{str(count)}.png",
                 "PNG",
                 progressive=True,
                 quality=100,
                 optimize=True)


Epoch 500:, Style Loss : 0.000033, Content Loss : 11.681394
Epoch 500:, Style Loss : 0.000032, Content Loss : 12.813873
Epoch 500:, Style Loss : 0.000035, Content Loss : 12.801293
Epoch 500:, Style Loss : 0.000033, Content Loss : 13.107387
Epoch 500:, Style Loss : 0.000035, Content Loss : 13.259162
Epoch 500:, Style Loss : 0.000034, Content Loss : 13.939225
Epoch 500:, Style Loss : 0.000035, Content Loss : 13.143072
Epoch 500:, Style Loss : 0.000036, Content Loss : 12.589658
Epoch 500:, Style Loss : 0.000032, Content Loss : 12.489207
Epoch 500:, Style Loss : 0.000034, Content Loss : 12.632652
Epoch 500:, Style Loss : 0.000036, Content Loss : 12.947369
Epoch 500:, Style Loss : 0.000036, Content Loss : 13.294169
Epoch 500:, Style Loss : 0.000038, Content Loss : 12.990697
Epoch 500:, Style Loss : 0.000032, Content Loss : 12.637298
Epoch 500:, Style Loss : 0.000031, Content Loss : 12.983603
Epoch 500:, Style Loss : 0.000033, Content Loss : 12.912018
Epoch 500:, Style Loss : 0.000038, Conte